# Trampoline Competition Scheduler

## Information
### Competitors
- First name of competitor
    - Stored as a string.
- Surname of competitor
    - Stored as a string.
- Preferred category of competitor
    - Stored as a boolean.
        - True: Female
        - False: Male
- Trampoline (TRA) competitor
    - Competing?
        - True or False.
    - Stored as an enumerator?
        - 1: Novice
        - 2: Intermediate
        - 3: Intervanced
        - 4: Advanced
        - 5: Elite
        - 6: Elite-Pro
- Double-mini trampoline (DMT) competitor
    - Competing?
        - True or False.
    - Stored as an int.
        - 1: Level 1
        - 2: Level 2
        - 3: Level 3
        - 4: Level 4
        - 5: Level 5
        - 6: Level 6
- Tumbling (TUM) competitor
    - Competing?
        - True or False.
    - Stored as an int.
        - 1: Level 1
        - 2: Level 2
        - 3: Level 3
        - 4: Level 4


### Judges
Level of trampoline (TRA) juding qualification
Chairperson


In [ ]:
class Level(Enum):
    NOVICE = 0
    INTERMEDIATE = 1

@dataclass
class Competitor:
  first_name: str
  surname: str
  level: Level

Hello, World!
